In [ ]:
from pyspark import SparkContext
sc = SparkContext("local[2]", "twitter_stream")

In [ ]:
execfile("shell_init.py")

In [ ]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark.streaming import StreamingContext

In [ ]:
ssc = StreamingContext(sc, 1)
zookeeper = ("localhost:2181")
kafka_stream = KafkaUtils.createStream(ssc, zookeeper, "twitter", {"twitter_api_stream":1})

In [ ]:
kafka_stream.pprint()

#ssc.awaitTermination(2)
ssc.start()

In [ ]:
ssc.stop()